# Lexicon Creation

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split
from PyClasses.Scrapers.PRPMScraper import PRPMScraper
from PyClasses.Preprocessing.PRPMCleaner import PRPMCleaner

class LexiconBuilder:
    def __init__(self, spark):
        self.spark = spark
        
    def get_reduced_words(self):
        try:
            file_path = "DE-prj/MR_WC_Result/part-00000"
            df = self.spark.read.text(file_path)
            df_split = df.select(split(df['value'], '\t').alias('word_count'))
            df_final = df_split.select(
                df_split['word_count'].getItem(0).alias('word'),  # First item is the word
                df_split['word_count'].getItem(1).cast('int').alias('count')  # Second item is the count, cast it to integer
            )
            return df_final.collect()
        except:
            return None

    def get_words_properties(self):
        df = self.get_reduced_words()
        PRPMscrap = PRPMScraper(PRPMCleaner())
        broadCastPRPMScrap_obj = spark.sparkContext.broadcast(PRPMscrap)
        def get_word_metadata(row):
            print(row)
            PRPMscrapper = PRPMScraper(PRPMCleaner())
            result = PRPMscrapper.findWordMetaData(row.word)
            print(result)
            return result
        #find all words meaning from prpm
        wordRows_rdd = self.spark.sparkContext.parallelize(df)
        result = wordRows_rdd.map(get_word_metadata).collect()
        return result
        
spark = SparkSession.builder.appName("DE-prj").getOrCreate()
lexiconBuilder = LexiconBuilder(spark)
try:
    hello = lexiconBuilder.get_words_properties()
finally:
    spark.stop()
    # Word, wordcount, synonym, antonym, definition, stemmed

/home/student/de-prj/de-venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3397
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
/home/student/de-prj/de-venv/lib/python3.10/site-packages/malaya/tokenizer.py:214: FutureWarning: Possible nested set at position 3927
  self.tok = re.compile(r'({})'.format('|'.join(pipeline)))
24/12/15 02:35:33 WARN Utils: Your hostname, Liang. resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/12/15 02:35:33 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
24/12/15 02:35:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java

In [2]:
print(len(hello))

TypeError: object of type 'PipelinedRDD' has no len()